In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, Dense, UpSampling2D, Conv2D, Conv2DTranspose, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import random
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import pandas as pd
from collections import defaultdict

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

2025-03-26 17:21:29.032688: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Step 1: Data Organization and Splitting
def organize_data(data_dir, test_size=0.2):
    """
    Organize data from folders and split into train/test sets
    
    Args:
        data_dir: Directory containing subfolders with cattle images
        test_size: Proportion of data for testing
    
    Returns:
        train_data, test_data: Lists of (image_path, label) tuples
    """
    all_data = []
    
    # Iterate through each subfolder
    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)
        
        if os.path.isdir(folder_path):
            # Get all images in the subfolder
            for img_name in os.listdir(folder_path):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(folder_path, img_name)
                    all_data.append((img_path, folder_name))
    
    # Split data into train and test sets
    train_data, test_data = train_test_split(all_data, test_size=test_size, stratify=[x[1] for x in all_data], random_state=42)
    
    return train_data, test_data

def create_data_directories(train_data, test_data, output_dir):
    """
    Create train and test directories with class subdirectories
    
    Args:
        train_data: List of (image_path, label) tuples for training
        test_data: List of (image_path, label) tuples for testing
        output_dir: Directory to create train and test folders
    """
    # Create main directories
    train_dir = os.path.join(output_dir, 'train')
    test_dir = os.path.join(output_dir, 'test')
    
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    
    # Create class subdirectories and copy images
    import shutil
    
    # Process training data
    for img_path, label in train_data:
        label_dir = os.path.join(train_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        
        # Copy the image
        shutil.copy(img_path, os.path.join(label_dir, os.path.basename(img_path)))
    
    # Process testing data
    for img_path, label in test_data:
        label_dir = os.path.join(test_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        
        # Copy the image
        shutil.copy(img_path, os.path.join(label_dir, os.path.basename(img_path)))
    
    return train_dir, test_dir

In [3]:
# Step 2: Create Data Generators
def create_data_generators(train_dir, test_dir, img_size=(224, 224), batch_size=32):
    """
    Create data generators for training and testing
    
    Args:
        train_dir: Directory containing training data
        test_dir: Directory containing testing data
        img_size: Input image dimensions
        batch_size: Batch size for training
    
    Returns:
        train_generator, validation_generator: Data generators
    """
    # Data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.1  # Use 10% of training data for validation
    )
    
    # Only rescaling for testing
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    # Training generator with validation split
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='input',  # For autoencoder, input is the target
        subset='training'
    )
    
    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='input',
        subset='validation'
    )
    
    # Test generator
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='input',
        shuffle=False
    )
    
    return train_generator, validation_generator, test_generator

In [4]:
# Step 3: Build Autoencoder with EfficientNet Encoder
def build_autoencoder(img_size=(224, 224, 3), latent_dim=128):
    """
    Build autoencoder with EfficientNet encoder and custom decoder
    
    Args:
        img_size: Input image dimensions
        latent_dim: Dimension of the latent space
    
    Returns:
        autoencoder: Complete autoencoder model
        encoder: Encoder part of the model for feature extraction
    """
    # Base EfficientNet model (encoder)
    base_model = EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_shape=img_size,
        pooling='avg'
    )
    
    # Freeze the pre-trained weights
    for layer in base_model.layers:
        layer.trainable = True
    
    # Input layer
    inputs = Input(shape=img_size)
    
    # Encoder
    x = base_model(inputs)
    x = Dense(latent_dim, activation='relu')(x)
    
    # Define encoder model for feature extraction
    encoder = Model(inputs, x, name='encoder')
    
    # Decoder
    x = Dense(7 * 7 * 64, activation='relu')(x)
    x = Reshape((7, 7, 64))(x)
    
    x = Conv2DTranspose(64, (3, 3), strides=2, padding='same', activation='relu')(x)  # 14x14
    x = Conv2DTranspose(32, (3, 3), strides=2, padding='same', activation='relu')(x)  # 28x28
    x = Conv2DTranspose(16, (3, 3), strides=2, padding='same', activation='relu')(x)  # 56x56
    x = Conv2DTranspose(8, (3, 3), strides=2, padding='same', activation='relu')(x)   # 112x112
    
    # Output layer
    outputs = Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='sigmoid')(x)  # 224x224
    
    # Define autoencoder model
    autoencoder = Model(inputs, outputs, name='autoencoder')
    
    return autoencoder, encoder

In [5]:
# Step 4: Implement GEM (Gradient Episodic Memory) - FIXED
class GradientEpisodicMemory:
    def __init__(self, memory_size=200):
        """
        Initialize GEM
        
        Args:
            memory_size: Maximum number of samples to store in memory
        """
        self.memory_size = memory_size
        self.memory_x = []
        self.memory_y = []
        self.memory_task_ids = []  # This was missing in the previous implementation
        self.task_memory_sizes = defaultdict(int)
    
    def add_example(self, x, y, task_id):
        """
        Add an example to episodic memory
        
        Args:
            x: Input data
            y: Target output
            task_id: Identifier for the task
        """
        # If memory is full, replace a random sample from the same task
        if len(self.memory_x) >= self.memory_size:
            task_indices = [i for i, task in enumerate(self.memory_task_ids) if task == task_id]
            if task_indices:
                replace_idx = random.choice(task_indices)
                self.memory_x[replace_idx] = x
                self.memory_y[replace_idx] = y
            else:
                # If no samples from this task, replace a random sample
                replace_idx = random.randrange(len(self.memory_x))
                self.memory_x[replace_idx] = x
                self.memory_y[replace_idx] = y
                self.memory_task_ids[replace_idx] = task_id
        else:
            # If memory is not full, just add the example
            self.memory_x.append(x)
            self.memory_y.append(y)
            self.memory_task_ids.append(task_id)
            self.task_memory_sizes[task_id] += 1
    
    def get_memory_batch(self, batch_size=32):
        """
        Get a batch of samples from memory
        
        Args:
            batch_size: Size of the batch to return
        
        Returns:
            memory_batch_x, memory_batch_y: Batch of samples from memory
        """
        if not self.memory_x:
            return None, None
        
        # Sample indices randomly
        indices = random.sample(range(len(self.memory_x)), min(batch_size, len(self.memory_x)))
        
        # Get the batch
        memory_batch_x = [self.memory_x[i] for i in indices]
        memory_batch_y = [self.memory_y[i] for i in indices]
        
        return np.array(memory_batch_x), np.array(memory_batch_y)

In [6]:
# Step 5: Training function with GEM
def train_with_gem(autoencoder, train_generator, validation_generator, gem, epochs=30, batch_size=32):
    """
    Train the autoencoder with GEM
    
    Args:
        autoencoder: Autoencoder model
        train_generator: Training data generator
        validation_generator: Validation data generator
        gem: GradientEpisodicMemory instance
        epochs: Number of epochs to train
        batch_size: Batch size
    
    Returns:
        history: Training history
    """
    # Compile the model
    autoencoder.compile(optimizer=Adam(learning_rate=1e-4), loss='mse')
    
    # Callbacks
    callbacks = [
        ModelCheckpoint('autoencoder_best.h5', save_best_only=True, monitor='val_loss'),
        EarlyStopping(patience=5, restore_best_weights=True),
        ReduceLROnPlateau(factor=0.5, patience=3, min_lr=1e-6)
    ]
    
    # Lists to store losses
    train_losses = []
    val_losses = []
    
    # Training loop
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")
        epoch_train_losses = []
        
        # Iterate through batches
        for batch_idx in range(len(train_generator)):
            # Get batch of data
            x_batch, _ = train_generator.next()
            
            # Train on the batch
            loss = autoencoder.train_on_batch(x_batch, x_batch)
            epoch_train_losses.append(loss)
            
            # Add examples to memory
            for i in range(len(x_batch)):
                gem.add_example(x_batch[i], x_batch[i], 0)  # Task ID is 0 for simplicity
            
            # If memory has samples, train on a batch from memory
            memory_x, memory_y = gem.get_memory_batch(batch_size)
            if memory_x is not None:
                autoencoder.train_on_batch(memory_x, memory_y)
            
            print(f"\rBatch {batch_idx+1}/{len(train_generator)} - Loss: {loss:.4f}", end="")
        
        # Compute validation loss
        val_loss = 0
        val_steps = 0
        for _ in range(len(validation_generator)):
            x_val, _ = validation_generator.next()
            val_batch_loss = autoencoder.evaluate(x_val, x_val, verbose=0)
            val_loss += val_batch_loss
            val_steps += 1
        
        val_loss /= val_steps
        
        # Store losses
        train_loss = np.mean(epoch_train_losses)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print(f"\nEpoch {epoch+1}/{epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")
    
    history = {
        'train_loss': train_losses,
        'val_loss': val_losses
    }
    
    return history

In [7]:
# Step 6: Feature extraction function
def extract_features(encoder, data_generator):
    """
    Extract features using the encoder
    
    Args:
        encoder: Encoder model
        data_generator: Data generator
    
    Returns:
        features: Extracted features
        labels: Corresponding labels
    """
    features = []
    labels = []
    
    # Iterate through all batches
    for i in range(len(data_generator)):
        # Get batch of data
        x_batch, _ = data_generator.next()
        
        # Extract features
        batch_features = encoder.predict(x_batch)
        
        # Store features and labels
        features.append(batch_features)
        
        # For labels, we'll use the directory names
        batch_labels = data_generator.classes[i*data_generator.batch_size:(i+1)*data_generator.batch_size]
        labels.extend(batch_labels)
    
    features = np.vstack(features)
    
    return features, labels

In [8]:
# Step 7: Visualization function
def visualize_results(history, features, labels):
    """
    Visualize training results and extracted features
    
    Args:
        history: Training history
        features: Extracted features
        labels: Corresponding labels
    """
    # Create a figure with subplots
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot training and validation loss
    axs[0].plot(history['train_loss'], label='Training Loss')
    axs[0].plot(history['val_loss'], label='Validation Loss')
    axs[0].set_title('Training and Validation Loss')
    axs[0].set_xlabel('Epoch')
    axs[0].set_ylabel('Loss')
    axs[0].legend()
    
    # Plot feature distribution using PCA
    from sklearn.decomposition import PCA
    
    pca = PCA(n_components=2)
    features_2d = pca.fit_transform(features)
    
    unique_labels = np.unique(labels)
    colors = plt.cm.jet(np.linspace(0, 1, len(unique_labels)))
    
    for i, label in enumerate(unique_labels):
        mask = labels == label
        axs[1].scatter(features_2d[mask, 0], features_2d[mask, 1], c=[colors[i]], label=f'Class {label}')
    
    axs[1].set_title('Feature Distribution (PCA)')
    axs[1].set_xlabel('Principal Component 1')
    axs[1].set_ylabel('Principal Component 2')
    axs[1].legend()
    
    plt.tight_layout()
    plt.savefig('training_results.png')
    plt.show()

In [ ]:
    # Replace with your actual data directory
data_dir = "../../dataset/All-images" 
output_dir = "./cattle_classification_output_all"

In [10]:
# Create output directory
os.makedirs(output_dir, exist_ok=True)
    
# Organize data
print("Organizing data...")
train_data, test_data = organize_data(data_dir)
    
# Create data directories
print("Creating data directories...")
train_dir, test_dir = create_data_directories(train_data, test_data, output_dir)
    
# Create data generators
print("Creating data generators...")
train_generator, validation_generator, test_generator = create_data_generators(train_dir, test_dir)

Organizing data...
Creating data directories...
Creating data generators...
Found 14309 images belonging to 1340 classes.
Found 840 images belonging to 1340 classes.
Found 3788 images belonging to 1340 classes.


In [11]:
# Step 8: Main execution function
def main(data_dir, output_dir):
    """
    Main execution function
    
    Args:
        data_dir: Directory containing cattle image subfolders
        output_dir: Directory to store processed data and results
    """

    
    # Build autoencoder
    print("Building autoencoder...")
    autoencoder, encoder = build_autoencoder()
    
    # Initialize GEM
    print("Initializing Gradient Episodic Memory...")
    gem = GradientEpisodicMemory()
    
    # Train with GEM
    print("Training with GEM...")
    history = train_with_gem(autoencoder, train_generator, validation_generator, gem)
    


In [ ]:
# Example usage
if __name__ == "__main__":

    
    main(data_dir, output_dir)

Building autoencoder...


2025-03-26 17:26:16.514352: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Initializing Gradient Episodic Memory...
Training with GEM...
Epoch 1/30
Batch 448/448 - Loss: 0.0468
Epoch 1/30 - Train Loss: 0.0374 - Val Loss: 0.1087
Epoch 2/30
Batch 448/448 - Loss: 0.0470
Epoch 2/30 - Train Loss: 0.0257 - Val Loss: 0.0341
Epoch 3/30
Batch 448/448 - Loss: 0.0247
Epoch 3/30 - Train Loss: 0.0223 - Val Loss: 0.0206
Epoch 4/30
Batch 448/448 - Loss: 0.0134
Epoch 4/30 - Train Loss: 0.0201 - Val Loss: 0.1065
Epoch 5/30
Batch 448/448 - Loss: 0.0192
Epoch 5/30 - Train Loss: 0.0184 - Val Loss: 0.1413
Epoch 6/30
Batch 448/448 - Loss: 0.0157
Epoch 6/30 - Train Loss: 0.0170 - Val Loss: 0.0439
Epoch 7/30
Batch 448/448 - Loss: 0.0150
Epoch 7/30 - Train Loss: 0.0159 - Val Loss: 0.0734
Epoch 8/30
Batch 448/448 - Loss: 0.0234
Epoch 8/30 - Train Loss: 0.0149 - Val Loss: 0.0179
Epoch 9/30
Batch 448/448 - Loss: 0.0081
Epoch 9/30 - Train Loss: 0.0141 - Val Loss: 0.0546
Epoch 10/30
Batch 448/448 - Loss: 0.0122
Epoch 10/30 - Train Loss: 0.0133 - Val Loss: 0.0370
Epoch 11/30
Batch 128/448 

In [ ]:
    # Extract features
print("Extracting features...")
test_features, test_labels = extract_features(encoder, test_generator)
    
    # Visualize results
print("Visualizing results...")
visualize_results(history, test_features, test_labels)
    
    # Save models
print("Saving models...")
autoencoder.save(os.path.join(output_dir, 'autoencoder_model.h5'))
encoder.save(os.path.join(output_dir, 'encoder_model.h5'))
    
print("Process completed successfully!")